In [53]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

In [ ]:
import pickle

In [54]:
# Re-load the data into pandas dataframes
actual_data = pd.read_csv('/Users/maryamwalidaliramadan/Desktop/Bachelor/original_data.csv')
actual_data.head(15)

,Is_this_student_female?,Does_the_student_study_Computer_Science?,Does_the_student_study_Pharmaceutical_Engineering?,Does_the_student_study_Architecture?,Is_this_student_in_their_second_year_of_study?,Is_this_student_in_their_fourth_year_of_study?,Has_this_student_ever_worked_as_a_Junior_Teaching_Assistant_before?,Is_this_student_currently_a_ranked_student?,Does_this_student_have_Dogs?,Does_this_student_have_a_Cat?,...,Does_this_student_currently_live_in_Obour?,Does_this_student_currently_live_in_Tagamoa?,Does_this_student_currently_live_in_Sheraton?,Does_this_student_currently_live_in_Abbasiya?,Does_this_student_currently_live_in_Heliopolis?,Has_this_student_ever_lived_in_a_different_city_or_country?,Does_this_student_have_One_sibling?,Does_this_student_have_two_siblings?,Does_this_student_have_Three_siblings?,Student_name
0,0,1,0,0,0,1,1,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,Iyad Elwy
1,1,1,0,0,0,1,0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,1.0,Janna Elalawy
2,1,1,0,0,0,1,0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,Jumana Amr
3,1,1,0,0,0,1,0,0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1,0.0,1.0,0.0,Joanna Samir
4,1,1,0,0,0,1,0,0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,1.0,Suzan Wael
5,1,1,0,0,0,1,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,Farida Abouward
6,0,1,0,0,0,1,1,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,Mohamed Eyad
7,1,0,0,1,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,Nour Riad
8,1,0,1,0,0,1,0,1,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1,0.0,0.0,1.0,Reem Mahmoud
9,1,0,1,0,0,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0,Nadin Khaled


In [55]:
# Handle missing values by replacing them with a number that represents 'unknown' or 'not applicable' (-1)
actual_data.fillna(2, inplace=True)
actual_data.head(15)

,Is_this_student_female?,Does_the_student_study_Computer_Science?,Does_the_student_study_Pharmaceutical_Engineering?,Does_the_student_study_Architecture?,Is_this_student_in_their_second_year_of_study?,Is_this_student_in_their_fourth_year_of_study?,Has_this_student_ever_worked_as_a_Junior_Teaching_Assistant_before?,Is_this_student_currently_a_ranked_student?,Does_this_student_have_Dogs?,Does_this_student_have_a_Cat?,...,Does_this_student_currently_live_in_Obour?,Does_this_student_currently_live_in_Tagamoa?,Does_this_student_currently_live_in_Sheraton?,Does_this_student_currently_live_in_Abbasiya?,Does_this_student_currently_live_in_Heliopolis?,Has_this_student_ever_lived_in_a_different_city_or_country?,Does_this_student_have_One_sibling?,Does_this_student_have_two_siblings?,Does_this_student_have_Three_siblings?,Student_name
0,0,1,0,0,0,1,1,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,Iyad Elwy
1,1,1,0,0,0,1,0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,1.0,Janna Elalawy
2,1,1,0,0,0,1,0,0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,Jumana Amr
3,1,1,0,0,0,1,0,0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1,0.0,1.0,0.0,Joanna Samir
4,1,1,0,0,0,1,0,0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,1,0.0,0.0,1.0,Suzan Wael
5,1,1,0,0,0,1,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,Farida Abouward
6,0,1,0,0,0,1,1,1,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0,Mohamed Eyad
7,1,0,0,1,1,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,Nour Riad
8,1,0,1,0,0,1,0,1,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1,0.0,0.0,1.0,Reem Mahmoud
9,1,0,1,0,0,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0,Nadin Khaled


In [56]:
# Split the actual data into features (X_dt) and labels (y_dt)
X_dt = actual_data.drop('Student_name', axis=1)
y_dt = actual_data['Student_name']

In [57]:

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dt, y_dt, test_size=0.2)

In [62]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [63]:
# Get feature importances from the trained random forest model
feature_importances = rf.feature_importances_

# Create a DataFrame to store feature names and their importances
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Choose a threshold for feature importance 
threshold = 0.02

# Select features above the threshold
selected_features = feature_importance_df[feature_importance_df['Importance'] > threshold]['Feature'].tolist()

# Subset your data based on selected features
X_train_subset = X_train[selected_features]
X_test_subset = X_test[selected_features]

# Train a new random forest model on the subset of features
rf_subset = RandomForestClassifier()
rf_subset.fit(X_train_subset, y_train)

# Make predictions on the test set using the subset model
y_pred_subset = rf_subset.predict(X_test_subset)

# Evaluate the performance of the subset model
# accuracy_subset = accuracy_score(y_test, y_pred_subset)
# print(f"Accuracy with a subset of features: {accuracy_subset}")

import joblib
# Save the subset model for later use
joblib.dump(rf_subset, "model_subset.pkl")

# Save the selected features to use in your Flask app
joblib.dump(selected_features, "model_cols_subset.pkl")

# Now you can use rf_subset to make predictions with a reduced set of questions
# Remember to preprocess new data in the same way as your training data


['model_cols_subset.pkl']

In [64]:
import joblib

# Load the content of the model_cols_subset.pkl file
model_columns = joblib.load("model_cols_subset.pkl")

# Print the content
print(model_columns)

['Has_this_student_ever_lived_in_a_different_city_or_country?', 'Is_Pop_the_favorite_genre_of_music_for_this_student?', 'Does_this_student_have_Dogs?', 'Does_this_student_enjoy_reading_books?', 'Is_Arabic_music_the_favorite_genre_of_music_for_this_student?', 'Is_this_student_participating_in_swimming?', 'Has_this_student_ever_worked_as_a_Junior_Teaching_Assistant_before?', 'Is_this_student_playing_Tennis?', 'Does_this_student_have_two_siblings? ', 'Does_this_student_have_a_Cat?', 'Does_the_student_study_Computer_Science?', 'Is_this_student_female?', 'Is_House_music_the_favorite_genre_of_music_for_this_student?', 'Has_this_student_ever_had_braces?', 'Does_this_student_currently_live_in_Nasr_City?', 'Does_this_student_have_One_sibling?', 'Are_Indie_Rock_and_Alternative_Rock_and_Jangle_Pop_the_favorite_genre_of_music_for_this_student?', 'Is_this_student_playing_Squash?', 'Does_this_student_use_an_Android?']


In [17]:
# cols = ['Does_this_student_have_Dogs?', 'Has_this_student_ever_had_braces?', 'Does_this_student_have_two_siblings? ', 'Is_this_student_a_night_owl?', 'Is_Tamer_Hosny_the_favorite_artist_for_this_student?', 'Is_this_student_a_morning_person?', 'Does_this_student_currently_live_in_Rehab_City?', 'Does_this_student_enjoy_reading_books?', 'Is_Pop_the_favorite_genre_of_music_for_this_student?', 'Is_this_student_currently_a_ranked_student?', 'Does_this_student_have_Three_siblings?', 'Has_this_student_ever_lived_in_a_different_city_or_country?', 'Does_this_student_have_One_sibling?', 'Is_this_student_playing_Football?', 'Are_Indie_Rock_and_Alternative_Rock_and_Jangle_Pop_the_favorite_genre_of_music_for_this_student?', 'Does_this_student_currently_live_in_Tagamoa?', 'Is_Arabic_music_the_favorite_genre_of_music_for_this_student?', 'Is_Indie_Rock_the_favorite_genre_of_music_for_this_student?']
# joblib.dump(cols, "model_cols_subset.pkl")

['model_cols_subset.pkl']

In [ ]:
# import joblib
# joblib.dump(rf_subset, "model.pkl")

['model.pkl']

In [ ]:
# cols = ["Is_this_student_female?", "Does_the_student_study_Computer_Science?",
# "Does_the_student_study_Pharmaceutical_Engineering?", "Does_the_student_study_Architecture?",
# "Is_this_student_in_their_second_year_of_study?", "Is_this_student_in_their_fourth_year_of_study?", "Has_this_student_ever_worked_as_a_Junior_Teaching_Assistant_before?",
# "Is_this_student_currently_a_ranked_student?", "Does_this_student_have_Dogs?", "Does_this_student_have_a_Cat?", "Is_Pop_the_favorite_genre_of_music_for_this_student?",
# "Is_Arabic_pop_the_favorite_genre_of_music_for_this_student?", "Is_Jazz_the_favorite_genre_of_music_for_this_student?" , "Is_Rock_the_favorite_genre_of_music_for_this_student?",
# "Are_Indie_Rock_and_ Alternative_Rock_and_Jangle_Pop_the_favorite_genre_of_music_for_this_student?", "Is_Country_music_the_favorite_genre_of_music_for_this_student?",
# "Is_Egyptian_pop_the_favorite_genre_of_music_for_this_student?", "Is_Phonk_music_the_favorite_genre_of_music_for_this_student?", "Is_Arabic_music_the_favorite_genre_of_music_for_this_student?",
# "Is_House_music_the_favorite_genre_of_music_for_this_student?", "Is_Indie_Rock_the_favorite_genre_of_music_for_this_student?", "Is_this_student_playing_Basketball?", "Is_this_student_participating_in_gym?", "Is_this_student_playing_Boxing_and_participating_in_gym?",
# "Is_this_student_playing_Pingpong_and_Padel?", "Is_this_student_participating_in_swimming?", "Is_this_student_playing_Tennis?", "Is_this_student_playing_Football?", "Is_this_student_playing_Squash?", "Is_this_student_playing_Fencing?",
# "Is_Amr_Diab_the_favorite_artist_for_this_student?", "Is_the_band_Small_Forward_the_favorite_Band_for_this_student?", "Is_Mohamed_Mounir_the_favorite_artist_for_this_student?", "Is_Tamer_Hosny_the_favorite_artist_for_this_student?",
# "Is_Taylor_Swift_the_favorite_artist_for_this_student?", "Is_Tom_Cruise_the_favorite_artist_for_this_student?", "Is_Travis_Scott_the_favorite_artist_for_this_student?", "Is_Myles_Smith_the_favorite_artist_for_this_student?",
# "Is_Post_Malone_the_favorite_artist_for_this_student?", "Does_this_student_enjoy_reading_books?", "Does_this_student_have_any_piercings?", "Does_this_student_have_both_piercings_and_tattoos?", "Has_this_student_ever_had_braces?", "Does_this_student_use_an_iPhone?", "Does_this_student_use_an_Android?",
# "Is_this_student_a_morning_person?", "Is_this_student_a_night_owl?", "Does_this_student_currently_smoke?", "Does_this_student_currently_live_in_Rehab_City?", "Does_this_student_currently_live_in_Maadi?", "Does_this_student_currently_live_in_Nasr_City?",
# "Does_this_student_currently_live_in_Obour?", "Does_this_student_currently_live_in_Tagamoa?", "Does_this_student_currently_live_in_Sheraton?", "Does_this_student_currently_live_in_Abbasiya?", "Does_this_student_currently_live_in_Heliopolis?", "Has_this_student_ever_lived_in_a_different_city_or_country?",
# "Does_this_student_have_One_sibling?", "Does_this_student_have_two_siblings?", "Does_this_student_have_Three_siblings?"]



# joblib.dump(cols, "model_cols.pkl")

['model_cols.pkl']

In [ ]:
# rf_pred = rf.predict(X_test)

testing part 👇🏻

In [ ]:
# Evaluate the model by comparing the actual and predicted student names
# rf_accuracy = accuracy_score(y_test, rf_pred)
# rf_accuracy
# rf_predicted_students = pd.DataFrame({'Actual_Student_Name': y_test, 'RF_Predicted_Student_Name': rf_pred})

# rf_predicted_students, rf_accuracy

In [ ]:
# # Assuming user_responses is a list of binary responses to questions
# user_responses = [1, 2, 0, 0,1]  # Example user responses

# # Make a prediction
# prediction = rf.predict([user_responses])

# print(f"The predicted character/object is: {prediction[0]}")


The predicted character/object is: Maryam


/Users/maryamwalidaliramadan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
